In [44]:
import pandas as pd
from IPython.display import HTML, clear_output


df = pd.read_json('regen.json')
if all(item in df.columns for item in ['curation_status', 'curation_message']):
  df.fillna(value={'curation_status': 0, 'curation_message': ''}, inplace=True)
# else:
# df['curation_status'] = 0
# df['curation_message'] = ''
df.tail(3)

,instruction,output,most_similar_instructions,avg_similarity_score,curation_status,curation_message
47,"Buatlah website ""Blog Pariwisata"" yang menampi...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website ""Forum Diskusi"" yang menyedi...",0.295151,0.0,
48,"Buatlah website kesehatan dengan nama ""Sehat S...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website ""Blog Pariwisata"" yang menam...",0.247180,0.0,
49,"Buatlah website restoran dengan judul ""Makanan...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website blog dengan judul ""Ruang Ins...",0.285178,0.0,


In [45]:
curation_df = pd.DataFrame(columns=['curation_status', 'curation_message'])

In [46]:
for idx, row in df[df['curation_status'] == 0].iterrows():
  last_idx = idx
  instruction, output, most_similar_instructions, avg_similarity_score, _, __ = row
  stop = False
  while not stop:
    clear_output(wait=True)
    display(HTML(output))
    
    print(f'''Instruction: {instruction}
    Output: {output}''')
    
    curator_input = input(f'{idx}. Is the data good? (yes/no/mark/stop):')
    curator_input = curator_input.strip()
    if curator_input == 'yes':
      curation_df.loc[idx] = {
        'curation_status': 1,
        'curation_message': '',
      }
      break
    elif curator_input == 'no':
      curation_df.loc[idx] = {
        'curation_status': -1,
        'curation_message': '',
      }
      break
    # Mark the data if you think the data can be fixed/improved manually
    elif curator_input == 'mark':
      suggestion = input('Suggestion: ')
      curation_df.loc[idx] = {
        'curation_status': 0,
        'curation_message': suggestion,
      }
      break
    elif curator_input == 'stop':
      stop = True
    else:
      print('Invalid input:', '"' + curator_input + '"')
  if stop:
    break
clear_output()

Instruction: Buatlah website pemerintah yang menampilkan informasi tentang pemerintahan negara. Desain website ini harus responsif dan memiliki navigasi yang mudah melalui menu dropdown pada navbar. Pada halaman utama terdapat sejarah negara dan visi misi pemerintahan. Bagian "Layanan Publik" menampilkan daftar layanan publik yang disediakan oleh pemerintah. Bagian "Berita dan Pengumuman" menampilkan berita dan pengumuman terbaru. Bagian "Kontak" menyediakan informasi kontak dan formulir untuk mengirim pesan kepada pemerintah.
    Output: <!DOCTYPE html>
<html lang="id">

<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Pemerintahan Negara</title>
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">
</head>

<body>
  <!-- HEADER -->
  <header class="bg-primary py-3

In [41]:
curation_df

,curation_status,curation_message
7,1,
8,1,
9,-1,
10,-1,
11,1,
12,-1,
13,-1,
14,-1,
15,-1,
16,1,


In [42]:
curated_df = curation_df.combine_first(df)[df.columns]
curated_df.iloc[curation_df.index]

,instruction,output,most_similar_instructions,avg_similarity_score,curation_status,curation_message
7,Buatlah website tutorial yang menampilkan daft...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website blog dengan judul ""Ruang Ins...",0.305832,1,
8,"Buatlah website ""Blog Fashion"" yang menyajikan...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website e-commerce ""Toko Furnitur"" y...",0.290212,1,
9,Buatlah website blog yang berisi artikel-artik...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...",{'Buatlah website blog yang menampilkan artike...,0.280568,-1,
10,Buatlah website organisasi yang menyediakan in...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...",{'Buatlah website tutorial yang menampilkan da...,0.316210,-1,
11,"Buatlah website ""Agensi Pariwisata"" yang menam...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website ""Reservasi Hotel"" yang memun...",0.290896,1,
12,Buatlah website portfolio pribadi dengan judul...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website blog berjudul ""Blog Remaja"" ...",0.292732,-1,
13,Buatlah website e-commerce untuk toko pakaian ...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...",{'Buatlah website portfolio pribadi dengan jud...,0.308134,-1,
14,Buatlah website blog dengan tema kesehatan dan...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...",{'Buatlah website e-commerce untuk toko pakaia...,0.326423,-1,
15,"Buatlah website ""Blog Pribadi"" yang menampilka...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website ""Agensi Pariwisata"" yang men...",0.325215,-1,
16,"Buatlah website ""Forum Diskusi"" yang menyediak...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website ""Blog Pribadi"" yang menampil...",0.295894,1,


In [43]:
curated_df[curated_df['curation_status'] != -1].to_json('regen-curated.json', orient="records")
print('regen-curated.json exported sucessfully!')

regen-curated.json exported sucessfully!


In [3]:
ok_df = pd.read_json('regen.json')
ok_df[['instruction', 'output']].to_json('webgen-dataset-2.json', orient='records')
print('webgen-dataset-2.json exported sucessfully!')

webgen-dataset-2.json exported sucessfully!
